### Что такое Langchain
** Langchain ** - Фреймворк для работы с языковыми моделями. Имеет очень большое кол-во инструментов для создания AI продукта

Основные компоненты
- **Models** - интерфейс для работы с LLM. Можно использовать API OpenAI, HuggingFace, Antropic и тд
- **Prompts** - форматирование промта и вывода модели
- **Indexs** - индексы нужны для векторных баз данных
- **Memory** - работа с состояниями в цепочках (можно сохранять ответы для быстродействия)
- **Chains** - Создание цепочек. Можно применить для разговоров, ответов на вопросы и др сценариев
- **Agents** - агенты дают доступ к различным источникам информации

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

course_api_key = getpass(prompt='Введи ключ:')

llm = ChatOpenAI(api_key=course_api_key
                ,model='gpt-4o-mini'
                ,base_url="https://aleron-llm.neuraldeep.tech/"
                )

### PromptTemplate
**PromptTemplate** - это инструмент для удобного форматирования своего шаблона промта

In [3]:
from langchain.prompts import PromptTemplate

#Пишем шаблон
template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: {query}

Answer: """

#Соединяем шаблон и запрос в шаблон промта
prompt_template = PromptTemplate(input_variables=['query']
                                 ,template=template)

#Собираем наш промт
prompt = prompt_template.format(query='Какая платформа онлайн курсов популярна в России?')


#Вывод
print(llm.invoke(prompt).content)

Популярной платформой онлайн курсов в России является Stepik.


### ChatPromptTemplate
**ChatPromptTemplate** - это инструмент для промтинга чатботов и систем Вопрос-Ответ

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    [
        ("system","Ты полезный AI-ассистент для проектировщика водоснабжения")
        ,("user", "Расскажи мне в паре предложений как проектировать - {topic}")
        ]
        )

# prompt_template.invoke({"topic": "канализацию"}) #Запихиваем в промт нужные данные по ключам
prompt = prompt_template.format_messages(topic="канализация")

print(llm.invoke(prompt).content)

Проектирование канализации включает в себя определение необходимых размеров труб, выбор материалов и планировку сети, чтобы обеспечить эффективный сбор и отвод сточных вод. Важно учитывать факторы, такие как уклон трубопроводов, наличие колодцев и выбор мест для обслуживания и ремонта системы, а также соответствие местным строительным нормам и стандартам.


### ChatPromptTemplate и техника Chain of Thought
**Chain of Thought** - техника при которой мы даем модели пример другой похожей задачи и даем ей цепочку рассуждений для правильного ответа

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate(
    [
        ("system","Ты полезный AI-ассистент, который режает задачи вдумчиво шаг за шагом")
        ,("human",'''Реши данную задачу: У меня есть стул, две картофелины, цветная капуста, 
                                        качан салата, два стола, капуста, две луковицы
                                        и три холодильника. Сколько у меня предметов мебели?''')
        ,('ai','''Сначала определим, что из перечисленного мебель: стол, стулья, холодильники.
                    Теперь посчитаем сколько их: стул 1, столы 2, холодильники 3
                    Посчитаем сумму: 1+2+3=6
                    Ответ: 6''')
        ,('human','Реши Реши данную задачу размышляя шаг зашагом: {question}')
    ]
)

question = """У меня есть стул, две картофелины, цветная капуста, качан салата, два стола, капуста, две луковицы
и три холодильника. Сколько у меня овощей?"""

prompt = prompt_template.format_messages(question=question)

print(llm.invoke(prompt).content)

Давайте разберем задачу и выделим овощи из перечисленных предметов.

Список предметов:
1. Стул (не овощ)
2. Две картофелины (овощи)
3. Цветная капуста (овощ)
4. Качан салата (овощ)
5. Два стола (не овощи)
6. Капуста (овощ)
7. Две луковицы (овощи)
8. Три холодильника (не овощи)

Теперь выделим овощи:
- Две картофелины: 2
- Цветная капуста: 1
- Качан салата: 1
- Капуста: 1
- Две луковицы: 2

Теперь посчитаем общее количество овощей:
- 2 (картофелины) + 1 (цветная капуста) + 1 (качан салата) + 1 (капуста) + 2 (луковицы) = 7

Ответ: У вас 7 овощей.


### FewShotPromptTemplate
**FewShotPromptTemplate** - техника при которой мы даем модели несколько примеров аналогов

In [7]:
from langchain.prompts import FewShotPromptTemplate

#Запись примеров
examples = [
    {"query": "Как дела?"
     ,"answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
     },
     {
         "query": "Сколько время?"
         ,"answer": "Самое время купить часы"
     }
]

#Создаем шаблон для примеров
example_template = """User: {query}
AI: {answer}
"""

#Создаем промпт с примерами
example_prompt = PromptTemplate(
    input_variables=['query','answer']
    ,template=example_template
)

#Теперь создаем префикс нашего промпта, который задает инструкцию модели
prefix = """Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:
"""

#Суффикс - поле для вопроса и ответа
suffix = """
User: {query}
AI: """

#Создаем сам few shot шаблон
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples
    ,example_prompt=example_prompt
    ,prefix=prefix
    ,suffix=suffix
    ,input_variables=["query"]
    ,example_separator="\n\n"
)

#Смотрим что получилось
query = "Почему падает снег?"

print(few_shot_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:


User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.


User: Сколько время?
AI: Самое время купить часы



User: Почему падает снег?
AI: 


In [ ]:
print(llm.invoke(few_shot_prompt_template.format(query=query)).content)

#Потому что снег решил, что потолки у всех слишком высокие, и пора немного понизить планку!

Потому что снег решил, что потолки у всех слишком высокие, и пора немного понизить планку!


### LengthBasedExampleSelector
**LengthBasedExampleSelector** - инструмент для задания примеров для модели, но с возможность динамически их включать/выключать

Это позволяет:
- не превышать контекстное окно
- экономить токены

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }, {
        "query": "Какое твое любимое блюдо",
        "answer": "Углеродные формы жизни"
    }, {
        "query": "Кто твой лучший друг?",
        "answer": "Siri. Мы любим с ней рассуждать о смысле жизни."
    }, {
        "query": "Что посоветуешь мне сделать сегодня?",
        "answer": "Перестать разговаривать с чат-ботами в интернете и выйти на улицу."
    }, {
        "query": "Какой твой любимый фильм?",
        "answer": "Терминатор, конечно."
    }
]

example_selector = LengthBasedExampleSelector(
    examples=examples
    ,example_prompt=example_prompt
    ,max_length=50   #Максимальная длина примера
)

#Создаем FewShot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector
    ,example_prompt=example_prompt
    ,prefix=prefix
    ,suffix=suffix
    ,input_variables=['query']
    ,example_separator='\n' #Как будем делить примеры
)

#Короткий запрос - можно уместить больше примеров
prompt = dynamic_prompt_template.format(query="Не могу вспомнить пароль")
print(prompt)

# ...
# User: Какое твое любимое блюдо
# AI: Углеродные формы жизни


# User: Не могу вспомнить пароль
# AI: 


Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.

User: Какое твое любимое блюдо
AI: Углеродные формы жизни


User: Не могу вспомнить пароль
AI: 


In [ ]:
query = '''Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?'''
print(dynamic_prompt_template.format(query=query))

# ...
# User: Сколько время?
# AI: Самое время купить часы.


# User: Я нахожусь во Владивостоке и хочу поехать заграницу.
# Я думаю в Китай или в Европу, во Францию или Испанию, например.
# Как мне лучше это сделать?
# AI:

#Длинный запрос - уже меньше примеров вмещаем

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.


User: Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?
AI: 


### StructuredOutputParser
**StructuredOutputParser** - инструмент для удобного вывода ответа модели

ResponseSchema - схема, которая говорит модели что и как включить в ответ. Это очень удобно, когда ты хочешь получить структурированный ответ в нужном тебе формате, содержащий нужную инфу и всё это в dict


In [21]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser



#Схема ответа - Является ли подарком
gift_schema = ResponseSchema(
    name='gift'
    ,description="""Был ли товар куплен в подарок кому-то другому?
    Ответь <True> если да и <False> если нет или неизвестно"""
)

#Схема ответа - Является ли подарком
delivery_days_schema = ResponseSchema(
    name='delivery_days'
    ,description="""Сколько дней потребовалось для доставки товара?
    Если эта информация не найдена, выведи -1"""
)

#Схема ответа - Является ли подарком
price_value_schema = ResponseSchema(
    name='price_value'
    ,description="""Извлеките любые предложения о стоимости или цене,
    и выведите их в виде списка Python, разделенного запятыми"""
)

#Список схем
response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [22]:
#Создаем парсер
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

#Получаем инструкции по форматированию ответа
format_instructions = output_parser.get_format_instructions()

#Наш текст для анализа
customer_review = """
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.
"""

#Создаем шаблон
review_template_2 = """\
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: {text}

{format_instructions}

"""

In [23]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review
                                  ,format_instructions=format_instructions)

response = llm.invoke(messages)
print(str(response))

#Получаем структурированный json с нашим форматов ответа

content='```json\n{\n\t"gift": "True",\n\t"delivery_days": 2,\n\t"price_value": ["Этот фен немного дороже, чем другие", "дополнительные функции того стоят"]\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 336, 'total_tokens': 381, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Bvf7Hb7aVUqgXj6ANpa5TqrJuBapT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--fa46cdee-3363-4dbf-9f09-c7564f24f588-0' usage_metadata={'input_tokens': 336, 'output_tokens': 45, 'total_tokens': 381, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

# {'gift': 'True',
#  'delivery_days': 2,
#  'price_value': ['Этот фен немного дороже, чем другие',
#   'дополнительные функции того стоят']}


#Теперь мы можем распарсить данные которые нам нужны из отзыва

{'gift': 'True',
 'delivery_days': 2,
 'price_value': ['Этот фен немного дороже, чем другие',
  'дополнительные функции того стоят']}